In [ ]:
#%load_ext autoreload
#%autoreload 2

In [1]:
#carga de datasets
from utils.DatasetStorage import Dataset
from utils.paths import *

#clasificadores
from utils.clasificacion import *

#adaptacion
from utils.adaptacion import msda_theano_pseudo_grid_search 

#otros
import os
import numpy as np
import pandas as pd
from sklearn.externals import joblib
import itertools

#variables para guardar los resultados
tipo = pruebas[1]
dataset_name = datasets[1]

Using Theano backend.
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


# Adaptación


## Creación de modelos de adaptación.

Para cada dominio se entrenan distintos modelos según los parámetros enviados.

Cada modelo es guardado en la ruta: models/twitter/msda/me1\_[dominio\_objetivo]_[numero_de_modelo].pkl

Todas las rutas son guardadas en una lista, la cual es almacenada en: models/twitter/msda/me1_models_paths.pkl

```python
paths_list = [ruta1, ..., ruta_n]
```

## Busqueda del mejor modelo por dominio

Por cada dominio se busca el mejor modelo de adaptación.

Esto se obtiene adaptando los datos de entrenamiento de cada dominio por cada modelo creado y realizando Grid-Search y Cross-Validation con estos datos.

El modelo que logre un mejor valor de Cross-Validation es considerado el mejor modelo para adaptar y queda guardado en un diccionario de la forma:

```python
best_models = {
    [dominio_1] = ruta,
    .
    .
    .
    [dominio_n] = ruta,
}
```

Este diccionario queda almacenado en la ruta:
    models/amazon/msda/me1_best_models.pkl

# Pruebas con el dataset Twitter (2000 Dimensiones)

In [2]:
dims = 2000

In [3]:
print tipo
print dataset_name
print dims
print data_path

msda
twitter
2000
data


In [4]:
# cargando dataset Amazon
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

Dataset already splitted


In [6]:
# se obtienen todos los valores X disponibles para realizar adaptacion
X = dataset_object.get_all_X()
X = np.asarray(X[: , :dims])

print X.shape

print "Todos los datos disponibles obtenidos"

(4635, 2000)
Todos los datos disponibles obtenidos


In [ ]:
models_paths = os.path.join(models_path, dataset_name, tipo, 'me1_%d_models_paths.pkl' % dims)
paths_list = []

# si existe el archivo con las rutas
# se carga la lista con las rutas
if os.path.exists(models_paths):
    print "Cargando rutas de modelos adaptados."
    paths_list = joblib.load(models_paths)
    print "Rutas cargadas en la variable 'paths_list'"
# si no
# se entrenan los modelos y se obtiene la lista con rutas
else:
    #se establecen los parametros para los modelos
    parametros = {
        'noises': [0.3, 0.5, 0.8],
        'layers': [1, 3, 5]
    }

    print "Creando modelos de adaptacion..."
    
    folder_path = os.path.join(models_path, dataset_name, tipo)
    prefix = "me1_%d_" % dims
        
    paths_list = msda_theano_pseudo_grid_search(X, parametros, folder_path, prefix)

    print "\nCreacion de modelos terminada\nGuardando rutas en %s" % models_paths
    joblib.dump(paths_list, models_paths)
    print "Rutas cargadas en la variable 'paths_list'"

Creando modelos de adaptacion...
	pr: 0.300 - l: 1
	Entrenando capa  1
	Entrenado en 0.89m

	Guardando modelo en models/twitter/msda/me1_2000_0.pkl
	pr: 0.300 - l: 3
	Entrenando capa  1
	Entrenando capa  2
	Entrenando capa  3
	Entrenado en 2.21m

	Guardando modelo en models/twitter/msda/me1_2000_1.pkl
	pr: 0.300 - l: 5
	Entrenando capa  1
	Entrenando capa  2
	Entrenando capa  3
	Entrenando capa  4
	Entrenando capa  5
	Entrenado en 3.44m

	Guardando modelo en models/twitter/msda/me1_2000_2.pkl
	pr: 0.500 - l: 1
	Entrenando capa  1
	Entrenado en 0.82m

	Guardando modelo en models/twitter/msda/me1_2000_3.pkl
	pr: 0.500 - l: 3
	Entrenando capa  1
	Entrenando capa  2
	Entrenando capa  3
	Entrenado en 2.11m

	Guardando modelo en models/twitter/msda/me1_2000_4.pkl
	pr: 0.500 - l: 5
	Entrenando capa  1
	Entrenando capa  2
	Entrenando capa  3
	Entrenando capa  4
	Entrenando capa  5
	Entrenado en 3.38m

	Guardando modelo en models/twitter/msda/me1_2000_5.pkl
	pr: 0.800 - l: 1
	Entrenando capa  1

In [ ]:
print "Modelos almacenados en:"
for ruta in paths_list:
    print "\t", ruta

Modelos almacenados en:
	models/twitter/msda/me1_2000_0.pkl
	models/twitter/msda/me1_2000_1.pkl
	models/twitter/msda/me1_2000_2.pkl
	models/twitter/msda/me1_2000_3.pkl
	models/twitter/msda/me1_2000_4.pkl
	models/twitter/msda/me1_2000_5.pkl
	models/twitter/msda/me1_2000_6.pkl
	models/twitter/msda/me1_2000_7.pkl
	models/twitter/msda/me1_2000_8.pkl


In [ ]:
best_models_paths = os.path.join(models_path, dataset_name, tipo, "me1_%d_best_models.pkl" % dims)
best_models = {}

if os.path.exists(best_models_paths):
    print "Cargando rutas de los mejores modelos..."
    best_models = joblib.load(best_models_paths)
    print "Rutas cargadas"  
else:
    print "Obteniendo mejores modelos..."
    for domain in domains:
        print domain
        i = 1
        best_score = 0
        best_path = None
        best_model = None
        
        for msda_model_path in paths_list:
            
            msda_model = joblib.load(msda_model_path)
            theano_model = msda_model['model']
            pr = msda_model['pr']
            n_layers = msda_model['l']

            print "\t%d) l=%d, pr=%.3f " % (i, msda_model['l'], msda_model['pr']),

            # se obtienen los datos del dominio
            X_tr = labeled[domain]['X_tr'][:, :dims].todense()
            y_tr = np.asarray(labeled[domain]['y_tr'].todense()).argmax(axis=1)

            # se adaptan los datos
            X_deep = theano_model.predict(X_tr)

            # se obtiene el mejor score con GridSearch
            new_clf = get_best_score(X_deep, y_tr, classifier='SVC', n_jobs=4)
            new_score = new_clf.best_score_
            print "score: %.4f" % new_score
            # se guarda si es el mejor para el modelo
            if new_score > best_score:
                best_score = new_score
                best_path = msda_model_path
                best_model = msda_model
            i = i+1

        #se guarda el mejor modelo para este dominio
        print "Mejor modelo: l=%d, pr=%.3f" % (best_model['l'], best_model['pr'])
        best_models[domain] = best_path
        
    
    # se guarda el diccionario con las mejores rutas
    joblib.dump(best_models, best_models_paths)
    print "Rutas guardadas en ", best_models_paths

Obteniendo mejores modelos...
rio2016
	1) l=1, pr=0.300  score: 0.8921
	2) l=3, pr=0.300  score: 0.8605
	3) l=5, pr=0.300 

/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with Standar

 score: 0.7455
	4) l=1, pr=0.500  score: 0.8884
	5) l=3, pr=0.500  score: 0.7742
	6) l=5, pr=0.500  score: 0.6656
	7) l=1, pr=0.800  score: 0.8835
	8) l=3, pr=0.800  score: 0.6379
	9) l=5, pr=0.800  score: 0.6332
Mejor modelo: l=1, pr=0.300
thevoice
	1) l=1, pr=0.300  score: 0.8947
	2) l=3, pr=0.300 

/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with Standar

 score: 0.8412
	3) l=5, pr=0.300 

...

 score: 0.7587
	4) l=1, pr=0.500  score: 0.8802
	5) l=3, pr=0.500  score: 0.7741
	6) l=5, pr=0.500 

/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with Standar

 score: 0.6824
	7) l=1, pr=0.800  score: 0.8572
	8) l=3, pr=0.800  score: 0.7152
	9) l=5, pr=0.800  score: 0.6984
Mejor modelo: l=1, pr=0.300
general
	1) l=1, pr=0.300 

/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/home/ubuntu/anaconda2/envs/env_memoria/lib/python2.7/site-packages/sklearn/svm/base.py:220: ConvergenceWarning: Solver terminated early (max_iter=50000).  Consider pre-processing your data with Standar

 score: 0.7278
	2) l=3, pr=0.300  score: 0.7110
	3) l=5, pr=0.300  score: 0.6789
	4) l=1, pr=0.500  score: 0.7200
	5) l=3, pr=0.500  score: 0.6723
	6) l=5, pr=0.500  score: 0.6340
	7) l=1, pr=0.800  score: 0.6961
	8) l=3, pr=0.800  score: 0.6303
	9) l=5, pr=0.800  score: 0.6030
Mejor modelo: l=1, pr=0.300
Rutas guardadas en  models/twitter/msda/me1_2000_best_models.pkl


In [ ]:
for dominio, ruta in best_models.items():
    print "%s - %s" % (dominio, ruta)

thevoice - models/twitter/msda/me1_2000_0.pkl
rio2016 - models/twitter/msda/me1_2000_0.pkl
general - models/twitter/msda/me1_2000_0.pkl


In [ ]:
#diccionario para mantener los dominios adaptados
adapted = {}

i = 0
for domain in domains:
    best_model_path = best_models[domain]
    
    if os.path.exists(best_model_path):
        print "Cargando mejor modelo para %s" % domain
        msda_model = joblib.load(best_model_path)
        theano_model = msda_model['model']
        
        print "\tAdaptando dominio"
        X_tr = labeled[domain]['X_tr'][:, :dims].todense()

        tr_reps = theano_model.predict(X_tr)

        adapted[domain] = {
            'X_tr': tr_reps
        }
        
        i = i+1
    else:
        print "\tGenerar mejor modelo para %s" % domain
        
print "%d/%d dominios adaptados" % (i, len(domains))

Cargando mejor modelo para rio2016
	Adaptando dominio
Cargando mejor modelo para thevoice
	Adaptando dominio
Cargando mejor modelo para general
	Adaptando dominio
3/3 dominios adaptados


## Pruebas de clasificación

In [ ]:
df = pd.DataFrame(columns=dataframe_columns)

i=0
tareas = len(domains)*(len(domains)-1)
pairs = list(itertools.permutations(domains, 2))

# por cada par posible para adaptar
for src, tgt in pairs:
    #se carga el mejor modelo para el dominio fuente
    ruta = best_models[src]
    modelo = joblib.load(ruta)
    msda_adapter = modelo['model']
     
    print "Tarea %d de %d" % (i+1, tareas)

    #baseline in-domain error
    #e_b(T,T)
    #entrenado en dominio tgt y probado en dominio tgt
    X_tr = labeled[tgt]['X_tr'][:, :dims].todense()
    y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)

    X_ts = labeled[tgt]['X_ts'][:, :dims].todense()
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    # se crean las rutas para cargar o crear los modelos
    model_name = "%d_%s.pkl" % (dims, tgt)
    model_path = os.path.join(models_path, dataset_name, "indomain", model_name)


    #Se realiza una clasificacion, estimando los parametros mediante cv
    svc = load_best_score(model_path, X_tr, y_tr)
    b_error = 1-svc.score(X_ts, y_ts)


    #############
    #### mSDA ###
    #############
    print "Adaptando dominios..."

    #entrenado en dominio src y probado en dominio tgt adaptados
    X_tr_a = adapted[src]['X_tr']
    y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)

    X_ts_a = msda_adapter.predict(X_ts)
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    # se calcula el transfer error
    svc_a = get_best_score(X_tr_a, y_tr, classifier='SVC', n_jobs=4)
    t_error = 1-svc_a.score(X_ts_a, y_ts)

    # transfer loss t
    # t_error - b_error
    t_loss = t_error - b_error

    tarea = src[0]+'->'+tgt[0]
    df.loc[i] = ['mSDA',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]

    i+=1
    
print "\nPruebas completadas"

Tarea 1 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 2 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 3 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 4 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 5 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 6 de 6
Cargando modelo existente.
Adaptando dominios...

Pruebas completadas


In [ ]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss
0,mSDA,r->t,rio2016,thevoice,7.362689,20.951705,13.589015
1,mSDA,r->g,rio2016,general,26.998127,44.610381,17.612254
2,mSDA,t->r,thevoice,rio2016,10.208711,12.477314,2.268603
3,mSDA,t->g,thevoice,general,26.998127,40.608619,13.610491
4,mSDA,g->r,general,rio2016,10.208711,17.286751,7.078040
5,mSDA,g->t,general,thevoice,7.362689,17.211174,9.848485


In [ ]:
new_scores_path = os.path.join(scores_path,dataset_name, tipo, "me1_%d.csv" % (dims))

print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."

Guardando en scores/twitter/msda/me1_2000.csv
Resultados guardados.


# Pruebas con el dataset Twitter (1000 Dimensiones)

In [ ]:
dims = 1000

In [ ]:
print tipo
print dataset_name
print dims
print data_path

msda
twitter
1000
data


In [ ]:
# cargando dataset Amazon
dataset_path = os.path.join(data_path, dataset_name+'.pkl')
dataset_object = Dataset().load(dataset_path)

dataset_object.split_dataset(test_size=0.2)

labeled = dataset_object.labeled
domains = dataset_object.domains

Dataset already splitted


In [ ]:
# se obtienen todos los valores X disponibles para realizar adaptacion
X = dataset_object.get_all_X()
X = np.asarray(X[: , :dims])

print X.shape

print "Todos los datos disponibles obtenidos"

(4635, 1000)
Todos los datos disponibles obtenidos


In [ ]:
models_paths = os.path.join(models_path, dataset_name, tipo, 'me1_%d_models_paths.pkl' % dims)
paths_list = []

# si existe el archivo con las rutas
# se carga la lista con las rutas
if os.path.exists(models_paths):
    print "Cargando rutas de modelos adaptados."
    paths_list = joblib.load(models_paths)
    print "Rutas cargadas en la variable 'paths_list'"
# si no
# se entrenan los modelos y se obtiene la lista con rutas
else:
    #se establecen los parametros para los modelos
    parametros = {
        'noises': [0.3, 0.5, 0.8],
        'layers': [1, 3, 5]
    }

    print "Creando modelos de adaptacion..."
    
    folder_path = os.path.join(models_path, dataset_name, tipo)
    prefix = "me1_%d_" % dims
        
    paths_list = msda_theano_pseudo_grid_search(X, parametros, folder_path, prefix)

    print "\nCreacion de modelos terminada\nGuardando rutas en %s" % models_paths
    joblib.dump(paths_list, models_paths)
    print "Rutas cargadas en la variable 'paths_list'"

Creando modelos de adaptacion...
	pr: 0.300 - l: 1
	Entrenando capa  1
	Entrenado en 0.07m

	Guardando modelo en models/twitter/msda/me1_1000_0.pkl
	pr: 0.300 - l: 3
	Entrenando capa  1
	Entrenando capa  2
	Entrenando capa  3
	Entrenado en 0.17m

	Guardando modelo en models/twitter/msda/me1_1000_1.pkl
	pr: 0.300 - l: 5
	Entrenando capa  1
	Entrenando capa  2
	Entrenando capa  3
	Entrenando capa  4
	Entrenando capa  5
	Entrenado en 0.27m

	Guardando modelo en models/twitter/msda/me1_1000_2.pkl
	pr: 0.500 - l: 1
	Entrenando capa  1
	Entrenado en 0.06m

	Guardando modelo en models/twitter/msda/me1_1000_3.pkl
	pr: 0.500 - l: 3
	Entrenando capa  1
	Entrenando capa  2
	Entrenando capa  3
	Entrenado en 0.16m

	Guardando modelo en models/twitter/msda/me1_1000_4.pkl
	pr: 0.500 - l: 5
	Entrenando capa  1
	Entrenando capa  2
	Entrenando capa  3
	Entrenando capa  4
	Entrenando capa  5
	Entrenado en 0.26m

	Guardando modelo en models/twitter/msda/me1_1000_5.pkl
	pr: 0.800 - l: 1
	Entrenando capa  1

In [ ]:
print "Modelos almacenados en:"
for ruta in paths_list:
    print "\t", ruta

Modelos almacenados en:
	models/twitter/msda/me1_1000_0.pkl
	models/twitter/msda/me1_1000_1.pkl
	models/twitter/msda/me1_1000_2.pkl
	models/twitter/msda/me1_1000_3.pkl
	models/twitter/msda/me1_1000_4.pkl
	models/twitter/msda/me1_1000_5.pkl
	models/twitter/msda/me1_1000_6.pkl
	models/twitter/msda/me1_1000_7.pkl
	models/twitter/msda/me1_1000_8.pkl


In [ ]:
best_models_paths = os.path.join(models_path, dataset_name, tipo, "me1_%d_best_models.pkl" % dims)
best_models = {}

if os.path.exists(best_models_paths):
    print "Cargando rutas de los mejores modelos..."
    best_models = joblib.load(best_models_paths)
    print "Rutas cargadas"  
else:
    print "Obteniendo mejores modelos..."
    for domain in domains:
        print domain
        i = 1
        best_score = 0
        best_path = None
        best_model = None
        
        for msda_model_path in paths_list:
            
            msda_model = joblib.load(msda_model_path)
            theano_model = msda_model['model']
            pr = msda_model['pr']
            n_layers = msda_model['l']

            print "\t%d) l=%d, pr=%.3f " % (i, msda_model['l'], msda_model['pr']),

            # se obtienen los datos del dominio
            X_tr = labeled[domain]['X_tr'][:, :dims].todense()
            y_tr = np.asarray(labeled[domain]['y_tr'].todense()).argmax(axis=1)

            # se adaptan los datos
            X_deep = theano_model.predict(X_tr)

            # se obtiene el mejor score con GridSearch
            new_clf = get_best_score(X_deep, y_tr, classifier='SVC', n_jobs=4)
            new_score = new_clf.best_score_
            print "score: %.4f" % new_score
            # se guarda si es el mejor para el modelo
            if new_score > best_score:
                best_score = new_score
                best_path = msda_model_path
                best_model = msda_model
            i = i+1

        #se guarda el mejor modelo para este dominio
        print "Mejor modelo: l=%d, pr=%.3f" % (best_model['l'], best_model['pr'])
        best_models[domain] = best_path
        
    
    # se guarda el diccionario con las mejores rutas
    joblib.dump(best_models, best_models_paths)
    print "Rutas guardadas en ", best_models_paths

Obteniendo mejores modelos...
rio2016
	1) l=1, pr=0.300  score: 0.8839
	2) l=3, pr=0.300  score: 0.8610
	3) l=5, pr=0.300  score: 0.7851
	4) l=1, pr=0.500  score: 0.8800
	5) l=3, pr=0.500  score: 0.8054
	6) l=5, pr=0.500  score: 0.6810
	7) l=1, pr=0.800  score: 0.8746
	8) l=3, pr=0.800  score: 0.7093
	9) l=5, pr=0.800  score: 0.6180
Mejor modelo: l=1, pr=0.300
thevoice
	1) l=1, pr=0.300  score: 0.8490
	2) l=3, pr=0.300  score: 0.7636
	3) l=5, pr=0.300  score: 0.6838
	4) l=1, pr=0.500  score: 0.8505
	5) l=3, pr=0.500  score: 0.7161
	6) l=5, pr=0.500  score: 0.6740
	7) l=1, pr=0.800  score: 0.8429
	8) l=3, pr=0.800  score: 0.6973
	9) l=5, pr=0.800  score: 0.6466
Mejor modelo: l=1, pr=0.500
general
	1) l=1, pr=0.300  score: 0.6682
	2) l=3, pr=0.300  score: 0.6634
	3) l=5, pr=0.300  score: 0.6332
	4) l=1, pr=0.500  score: 0.6687
	5) l=3, pr=0.500  score: 0.6471
	6) l=5, pr=0.500  score: 0.5854
	7) l=1, pr=0.800  score: 0.6640
	8) l=3, pr=0.800  score: 0.6013
	9) l=5, pr=0.800  score: 0.561

In [ ]:
for dominio, ruta in best_models.items():
    print "%s - %s" % (dominio, ruta)

thevoice - models/twitter/msda/me1_1000_3.pkl
rio2016 - models/twitter/msda/me1_1000_0.pkl
general - models/twitter/msda/me1_1000_3.pkl


In [ ]:
#diccionario para mantener los dominios adaptados
adapted = {}

i = 0
for domain in domains:
    best_model_path = best_models[domain]
    
    if os.path.exists(best_model_path):
        print "Cargando mejor modelo para %s" % domain
        msda_model = joblib.load(best_model_path)
        theano_model = msda_model['model']
        
        print "\tAdaptando dominio"
        X_tr = labeled[domain]['X_tr'][:, :dims].todense()

        tr_reps = theano_model.predict(X_tr)

        adapted[domain] = {
            'X_tr': tr_reps
        }
        
        i = i+1
    else:
        print "\tGenerar mejor modelo para %s" % domain
        
print "%d/%d dominios adaptados" % (i, len(domains))

Cargando mejor modelo para rio2016
	Adaptando dominio
Cargando mejor modelo para thevoice
	Adaptando dominio
Cargando mejor modelo para general
	Adaptando dominio
3/3 dominios adaptados


## Pruebas de clasificación

In [ ]:
df = pd.DataFrame(columns=dataframe_columns)

i=0
tareas = len(domains)*(len(domains)-1)
pairs = list(itertools.permutations(domains, 2))

# por cada par posible para adaptar
for src, tgt in pairs:
    #se carga el mejor modelo para el dominio fuente
    ruta = best_models[src]
    modelo = joblib.load(ruta)
    msda_adapter = modelo['model']
     
    print "Tarea %d de %d" % (i+1, tareas)

    #baseline in-domain error
    #e_b(T,T)
    #entrenado en dominio tgt y probado en dominio tgt
    X_tr = labeled[tgt]['X_tr'][:, :dims].todense()
    y_tr = np.asarray(labeled[tgt]['y_tr'].todense()).argmax(axis=1)

    X_ts = labeled[tgt]['X_ts'][:, :dims].todense()
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    # se crean las rutas para cargar o crear los modelos
    model_name = "%d_%s.pkl" % (dims, tgt)
    model_path = os.path.join(models_path, dataset_name, "indomain", model_name)


    #Se realiza una clasificacion, estimando los parametros mediante cv
    svc = load_best_score(model_path, X_tr, y_tr)
    b_error = 1-svc.score(X_ts, y_ts)


    #############
    #### mSDA ###
    #############
    print "Adaptando dominios..."

    #entrenado en dominio src y probado en dominio tgt adaptados
    X_tr_a = adapted[src]['X_tr']
    y_tr = np.asarray(labeled[src]['y_tr'].todense()).argmax(axis=1)

    X_ts_a = msda_adapter.predict(X_ts)
    y_ts = np.asarray(labeled[tgt]['y_ts'].todense()).argmax(axis=1)

    # se calcula el transfer error
    svc_a = get_best_score(X_tr_a, y_tr, classifier='SVC', n_jobs=4)
    t_error = 1-svc_a.score(X_ts_a, y_ts)

    # transfer loss t
    # t_error - b_error
    t_loss = t_error - b_error

    tarea = src[0]+'->'+tgt[0]
    df.loc[i] = ['mSDA',tarea,src,tgt,b_error*100,t_error*100, t_loss*100]

    i+=1
    
print "\nPruebas completadas"

Tarea 1 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 2 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 3 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 4 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 5 de 6
Cargando modelo existente.
Adaptando dominios...
Tarea 6 de 6
Cargando modelo existente.
Adaptando dominios...

Pruebas completadas


In [ ]:
df

,Adaptacion,Tarea,Fuente,Objetivo,Baseline error,Transfer error,Transfer loss
0,mSDA,r->t,rio2016,thevoice,19.045928,23.129735,4.083807
1,mSDA,r->g,rio2016,general,30.407980,43.450889,13.042909
2,mSDA,t->r,thevoice,rio2016,7.372958,13.271325,5.898367
3,mSDA,t->g,thevoice,general,30.407980,41.320104,10.912125
4,mSDA,g->r,general,rio2016,7.372958,13.362069,5.989111
5,mSDA,g->t,general,thevoice,19.045928,21.496212,2.450284


In [ ]:
new_scores_path = os.path.join(scores_path,dataset_name, tipo, "me1_%d.csv" % (dims))

print "Guardando en %s" % new_scores_path
df.to_csv(new_scores_path, columns=df.columns)
print "Resultados guardados."

Guardando en scores/twitter/msda/me1_1000.csv
Resultados guardados.
